# Pipeline Optimisé - 🚀 Amélioration +1% Accuracy

**Objectif**: Passer de 50.66% à 51.91%+

Ce pipeline intègre:
- ✅ Hyperparameter tuning (Optuna)
- ✅ Features avancées (20+ nouvelles features)
- ✅ Stacking ensemble (meta-learning)
- ✅ Threshold optimization
- ✅ Weighted ensemble

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Imports des modules
from data_loading import load_data, explore_data
from data_preparation import prepare_data, split_temporal_data, prepare_features_for_modeling
from feature_engineering import create_all_features
from exploratory_analysis import analyze_correlations, plot_correlations
from model_training import train_base_models, create_results_dataframe
from cross_validation import perform_time_series_cross_validation, plot_cv_results, get_feature_importance
from predictions import train_final_model
from optimization import optimize_hyperparameters, train_lgbm_optimized, quick_hyperparameter_search
from stacking_ensemble import multi_level_stacking, weighted_ensemble
from threshold_optimization import find_optimal_threshold, compare_thresholds
from utils import plot_model_comparison, print_summary_report

print("✅ Tous les modules importés - Pipeline Optimisé")
print("=" * 70)

## 📥 Étape 1: Chargement et Exploration

In [ ]:
X_train, y_train, X_test, sample_submission = load_data()
print(f"✓ Données chargées")
print(f"  Train: {X_train.shape} | Test: {X_test.shape}")

## 🔧 Étape 2: Préparation et Features (AMÉLIORÉES)

In [ ]:
train_data, X_test = prepare_data(X_train, y_train, X_test)
train_data, X_test, all_features = create_all_features(train_data, X_test)

print(f"\n✨ {len(all_features)} features créées (dont 20+ avancées)")

# Préparer X, y
X = train_data[all_features].fillna(0)
y = (train_data['target'] > 0).astype(int)
X_test_features = X_test[all_features].fillna(0)

# Split temporel
train_mask, test_mask = split_temporal_data(train_data, 0.7)
X_train_split, y_train_split = X[train_mask], y[train_mask]
X_val_split, y_val_split = X[test_mask], y[test_mask]

print(f"\n✓ Données préparées:")
print(f"  Train split: {X_train_split.shape}")
print(f"  Val split: {X_val_split.shape}")

## ⚙️ Étape 3: Hyperparameter Tuning (OPTUNA)

In [ ]:
print("\n" + "="*70)
print("🔍 HYPERPARAMETER OPTIMIZATION (rapide: 10 trials)")
print("="*70)

# Option 1: Quick search (rapide - 2min)
best_params = quick_hyperparameter_search(X_train_split, y_train_split, X_val_split, y_val_split)

# Option 2: Full Optuna optimization (lent - 15min, décommenter si besoin)
# best_params = optimize_hyperparameters(X_train_split, y_train_split, X_val_split, y_val_split, n_trials=50)

## 🔄 Étape 4: Cross-Validation avec Hyperparamètres Optimisés

In [ ]:
cv_scores, cv_models, mean_score, std_score = perform_time_series_cross_validation(
    X, y, train_data, all_features
)

# Afficher les résultats
print(f"\n✅ CV Score: {mean_score:.4f} ± {std_score:.4f}")

# Feature importance
feature_importance_df = get_feature_importance(cv_models, all_features, top_n=30)

## 🎯 Étape 5: Stacking Ensemble (Meta-Learning)

In [ ]:
# Entraîner le modèle final avec meilleurs hyperparamètres
lgbm_final = train_final_model(X, y, all_features)

# Stacking multi-niveau
print("\n" + "="*70)
pred_stacking, stacking_acc, meta_model = multi_level_stacking(
    cv_models, lgbm_final, X_train_split, X_val_split, X_test_features, 
    y_train_split, y_val_split
)

print(f"\n✅ Stacking Accuracy: {stacking_acc:.4f}")

# Ensemble pondéré (basé sur CV scores)
print("\n" + "="*70)
pred_weighted = weighted_ensemble(cv_models, lgbm_final, X_test_features, cv_scores)

## 🎪 Étape 6: Threshold Optimization

In [ ]:
print("\n" + "="*70)
print("🎯 OPTIMISATION DU SEUIL")
print("="*70)

# Obtenir les prédictions probabilistes pour le stacking
y_val_pred_proba_stacking = meta_model.predict(np.column_stack([
    cv_models[i].predict(X_val_split) for i in range(len(cv_models))
] + [lgbm_final.predict(X_val_split)]))

# Trouver le meilleur seuil
threshold_result = find_optimal_threshold(y_val_split.values, y_val_pred_proba_stacking)
optimal_threshold = threshold_result['threshold']

print(f"\n✓ Meilleur seuil trouvé: {optimal_threshold:.3f}")

# Comparer plusieurs seuils
print("\n📊 Comparaison de plusieurs seuils:")
thresholds_comparison = compare_thresholds(
    y_val_split.values, 
    y_val_pred_proba_stacking,
    thresholds=[0.40, 0.45, 0.50, 0.55, 0.60]
)

## 💾 Étape 7: Génération des Soumissions

In [ ]:
print("\n" + "="*70)
print("📋 SOUMISSIONS GÉNÉRÉES")
print("="*70)

# 1. Stacking avec seuil optimisé
submission_stacking = sample_submission.copy()
submission_stacking['target'] = pred_stacking
submission_stacking.to_csv('submission_stacking_optimized.csv')
print(f"\n✓ submission_stacking_optimized.csv")
print(f"  Positif: {pred_stacking.sum():,} ({pred_stacking.mean():.2%})")

# 2. Weighted ensemble
submission_weighted = sample_submission.copy()
submission_weighted['target'] = pred_weighted
submission_weighted.to_csv('submission_weighted_ensemble.csv')
print(f"\n✓ submission_weighted_ensemble.csv")
print(f"  Positif: {pred_weighted.sum():,} ({pred_weighted.mean():.2%})")

# 3. Stacking avec seuil optimal
print(f"\n✓ submission_stacking_threshold_{optimal_threshold:.3f}.csv")
pred_stacking_optimized = (y_val_pred_proba_stacking > optimal_threshold).astype(int)
# Appliquer au test set
y_test_pred_proba_stacking = meta_model.predict(np.column_stack([
    cv_models[i].predict(X_test_features) for i in range(len(cv_models))
] + [lgbm_final.predict(X_test_features)]))
pred_test_optimized = (y_test_pred_proba_stacking > optimal_threshold).astype(int)

submission_optimized = sample_submission.copy()
submission_optimized['target'] = pred_test_optimized
submission_optimized.to_csv(f'submission_stacking_threshold_{optimal_threshold:.3f}.csv')
print(f"  Positif: {pred_test_optimized.sum():,} ({pred_test_optimized.mean():.2%})")

print(f"\n" + "="*70)
print("🎯 RECOMMANDATION: Soumettre 'submission_stacking_optimized.csv'")
print("="*70)